# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
import winsound

In [2]:
data = get_date_list(data)
data = read_data(data)

Record num : 139426


In [3]:
for data['i'] in tqdm(range(0, data['df_len'])):
    
    data['df']['DateTime_frmt'][data['i']] = dt.datetime.strptime(data['df']['DateTime'][data['i']].split(".")[0],"%Y%m%d %H:%M:%S")
    
    data['ask'] = data['df']['Ask'][data['i']]
    data['bid'] = data['df']['Bid'][data['i']]  
    
    data['tick'] = (data['df']['Ask'][data['i']] + data['df']['Bid'][data['i']])/2
    data['df']['tick'][data['i']] = data['tick']
        
    # sema before after loops --------------------------------
    if len(data['sema_list']) < data['sema_len']:
        data =  before_sema(data)

    if len(data['sema_list']) == data['sema_len']:
        data = after_sema(data)                 
    # ----------------------------------------------------------
    
        # sema before after loops --------------------------------
    if len(data['sema_list']) < data['sema_len']:
        data =  before_sema(data)

    if len(data['sema_list']) == data['sema_len']:
        data = after_sema(data)                 
    # ----------------------------------------------------------


#     # lema before after loops --------------------------------
#     if len(data['lema_list']) < data['lema_len']:
#         data = before_lema(data)        
#         continue

#     if len(data['lema_list']) == data['lema_len']:
#         data = after_lema(data)                  
#     # ----------------------------------------------------------
    
    data = get_ohlc(data)    
    
#     data = get_position(data)
#     if not data['position']:
#         continue
        
#     # Get Dirs --------------------------------
#     if len(data['dir_list']) < 2:
#         data['dir_list'].append(data['position'])   
#         continue

#     elif len(data['dir_list']) == 2:
#         data = get_cross_dir(data)
#     # ----------------------------------------------------------  
    
    data = get_trend_fwd(data)
    data = trend_angle_check(data)

100%|█████████████████████████████████| 139426/139426 [17:49<00:00, 130.34it/s]


In [4]:
data['df']

,DateTime,Bid,Ask,tick,DateTime_frmt,sema,lema,position,direction,to_order,h_line_angle,l_line_angle,trend_angle,h_trend_calc_spot,l_trend_calc_spot,h,l
0,20210106 00:00:00.067,1.22985,1.22990,1.229875,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20210106 00:00:00.169,1.22987,1.22992,1.229895,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20210106 00:00:00.270,1.22988,1.22993,1.229905,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20210106 00:00:00.372,1.22989,1.22994,1.229915,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20210106 00:00:00.980,1.22988,1.22993,1.229905,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139421,20210106 21:00:02.905,1.23245,1.23250,1.232475,2021-01-06 21:00:02,1.232401,NaN,NaN,NaN,NaN,15.272674,16.615901,up,1.234306,1.233189,NaN,NaN
139422,20210106 21:00:03.240,1.23244,1.23249,1.232465,2021-01-06 21:00:03,1.232402,NaN,NaN,NaN,NaN,15.272674,16.615901,up,1.234306,1.233189,NaN,NaN
139423,20210106 21:00:04.213,1.23245,1.23250,1.232475,2021-01-06 21:00:04,1.232403,NaN,NaN,NaN,NaN,15.272674,16.615901,up,1.234307,1.233189,NaN,NaN
139424,20210106 21:00:04.517,1.23247,1.23250,1.232485,2021-01-06 21:00:04,1.232404,NaN,NaN,NaN,NaN,15.272674,16.615901,up,1.234307,1.233190,NaN,NaN


In [5]:
num_rec = len(data["df"])
hours = np.round((list(data['df']['DateTime_frmt'])[-1] - list(data['df']['DateTime_frmt'])[0]).total_seconds() / 60 / 60 , 2)
recsphour = int(num_rec/hours)

print(f'Records   : {num_rec}')
print(f'Hours     : {hours}')
print(f'Recs/hour : {recsphour}')
print(f'Recs/min  : {int(recsphour/60)}')
print(f'Recs/sec  : {np.round(recsphour/3600, 1)}')

data['df'].head()

Records   : 139426
Hours     : 21.0
Recs/hour : 6639
Recs/min  : 110
Recs/sec  : 1.8


,DateTime,Bid,Ask,tick,DateTime_frmt,sema,lema,position,direction,to_order,h_line_angle,l_line_angle,trend_angle,h_trend_calc_spot,l_trend_calc_spot,h,l
0,20210106 00:00:00.067,1.22985,1.22990,1.229875,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20210106 00:00:00.169,1.22987,1.22992,1.229895,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20210106 00:00:00.270,1.22988,1.22993,1.229905,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20210106 00:00:00.372,1.22989,1.22994,1.229915,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20210106 00:00:00.980,1.22988,1.22993,1.229905,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
plot_graph(data)

In [7]:
data['df'].to_csv('data/temp.csv')

In [8]:
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)

In [11]:
send_telegram_message(f'Run Complete at : {dt.datetime.now()}')